<a href="https://colab.research.google.com/github/Lorddickenstein/FSLRwithNLP/blob/main/Application/NLP/CFG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import nltk
from nltk import Tree
from subprocess import *
grammar1 = nltk.CFG.fromstring("""
  S -> QP | VP SP | QP SP | O VP | WP | O JJ 
  SP -> PRP NN | PRP IN | NN | PRP |
  QP -> WP NN | WRB A2 | WP | WRB
  O -> SP A1 | SP A2 A3 | SP
  VP -> VB RB | VB WRB | VB TO | QP
  A1 -> "is"
  A2 -> "are"
  A3 -> "am"
  IN -> "from"
  JJ -> "good"
  WP -> "what" | "when"
  WDT -> "which"
  WRB -> "where" | "how"
  NN -> "name" | "okay" | "office" | "work"
  PRP -> "you" | "i"
  RB -> "here" 
  TO -> "to"
  VB -> "live" | "go"
  """)


# (S (QP (WP What)) (SP (PRP you) (NN name)))
grammar2 = nltk.CFG.fromstring("""
  S -> QP | VP SP | QP A1 SP | O SP | O VBP
  SP -> NN | PRP NN 
  QP ->  WP
  WP -> "what"
  A1 -> "is"
  NN -> "name"
  PRP -> "your"
  """)

grammar4 = nltk.CFG.fromstring("""
  S -> A
  A -> QP SP  
  SP -> NN
  QP -> WP O
  O -> PRP
  WP -> "what"
  PRP -> "your"
  NN -> "name"
  A1 -> "is"
  """)

grammar7= nltk.CFG.fromstring("""
  S -> A
  A -> QP SP  
  SP -> NN
  QP -> WP O
  O -> PRP
  WP -> "what"
  PRP -> "your"
  NN -> "name"
  A1 -> "is"
  """)
 	
grammar3 = nltk.CFG.fromstring("""
  S -> NP VP
  VP -> V NP | V NP PP
  PP -> P NP
  V -> "saw" | "ate" | "walked"
  NP -> "John" | "Mary" | "Bob" | Det N | Det N PP
  Det -> "a" | "an" | "the" | "my"
  N -> "man" | "dog" | "cat" | "telescope" | "park"
  P -> "in" | "on" | "by" | "with"
  """)

grammar6 = nltk.CFG.fromstring("""
  S -> NP VP
  PP -> P NP
  NP -> Det N | Det N PP
  VP -> V NP | VP PP
  Det -> 'DT'
  N -> 'NN'
  V -> 'VBZ'
  P -> 'PP'
  """)
# What's your name (you name what?) - (S (O (SP (PRP you) (NN name))) (VBP (QP (WP what))))
# I'm good (I good) - (S (O (SP (PRP i))) (JJ good))
# Where you from (you from where) -> (S (O (SP (PRP you) (IN from))) (VBP (QP (WRB where))))
# I live here -> (S (O (SP (PRP i))) (VBP (VB live) (RB here)))
# How are you (How you?) -> (S (QP (WRB how)) (SP (PRP you)))
# Where do you live? (you, live, where?) -> (S (O (SP (PRP you))) (VBP (VB live) (WRB where)))
# Are you okay? (you, okay) ->(S (O (SP (PRP you) (NN okay))))
# My name is (I-Me, name) -> (S (O (SP (PRP i) (NN name))))
# Go to the Office (Go, to, Office) -> (S (VP (VB go) (TO to)) (SP (NN office)))
# Go to work (Go, to, work) -> (S (VBP (VB go) (TO to)) (SP (NN work)))

In [ ]:
grammar1.productions()

[S -> QP,
 S -> VP SP,
 S -> QP SP,
 S -> O VBP,
 S -> WP,
 S -> O JJ,
 S -> VBP SP,
 SP -> PRP NN,
 SP -> PRP IN,
 SP -> NN,
 SP -> PRP,
 SP -> ,
 QP -> WP NN,
 QP -> WRB A2,
 QP -> WP,
 QP -> WRB,
 O -> SP A1,
 O -> SP A2 A3,
 O -> SP,
 VBP -> VB RB,
 VBP -> VB WRB,
 VBP -> VB TO,
 VBP -> QP,
 A1 -> 'is',
 A2 -> 'are',
 A3 -> 'am',
 IN -> 'from',
 JJ -> 'good',
 WP -> 'what',
 WP -> 'when',
 WDT -> 'which',
 WRB -> 'where',
 WRB -> 'how',
 NN -> 'name',
 NN -> 'okay',
 NN -> 'office',
 NN -> 'work',
 PRP -> 'you',
 PRP -> 'i',
 RB -> 'here',
 TO -> 'to',
 VB -> 'live',
 VB -> 'go']

In [ ]:
import random


d = {}
 
d["s"]=[["a", "b"]]
d["a"]=[["what", "e"],["when", "e"]]
d["b"]=[["c", "d"]]
d["c"]=[["you"]]
d["d"]=[["name"],["bday"]]
d["e"]=[["is"]]

expansion = []
seed = "s"


def expand(start, expansion):
    if start in d:
    #Grab one possible expansion
        possible = d[start]
        random_expansion = random.choice(possible)
    # Call this method again with the current element
        for i in range(len(random_expansion)):
            expand(random_expansion[i], expansion);
 
  # if the rule wasn't found, then it's a terminal:
  # append the string to the expansion
    else:
        expansion.append(start)

expand(seed, expansion)

print(expansion)

['when', 'is', 'you', 'bday']


In [14]:
import re

class ParseError(Exception):
    pass

# Tokenize a string.
# Tokens yielded are of the form (type, string)
# Possible values for 'type' are '(', ')' and 'WORD'
def tokenize(s):
    toks = re.compile(' +|[A-Za-z]+|[()]')
    for match in toks.finditer(s):
        s = match.group(0)
        if s[0] == ' ':
            continue
        if s[0] in '()':
            yield (s, s)
        else:
            yield ('word', s)


# Parse once we're inside an opening bracket.
def parse_inner(toks):
    ty, name = next(toks)
    if ty != 'word': raise ParseError
    children = []
    while True:
        ty, s = next(toks)
        if ty == '(':
            children.append(parse_inner(toks))
        elif ty == ')':
            return (name, children)

# Parse this grammar:
# ROOT ::= '(' INNER
# INNER ::= WORD ROOT* ')'
# WORD ::= [A-Za-z]+
def parse_root(toks):
    ty, _ = next(toks)
    if ty != '(': raise ParseError
    return parse_inner(toks)

def show_children(tree):
    name, children = tree
    if not children: return
    print ("%s -> %s" % (name, ' '.join(child[0] for child in children)))
    for child in children:
        show_children(child)

example = '(S (O (SP (PRP you) (NN name))) (VP (QP (WP what))))'
show_children(parse_root(tokenize(example)))

S -> O VP
O -> SP
SP -> PRP NN
VP -> QP
QP -> WP


In [9]:

text = "you name what"
sent = text.split()
# rd_parser = nltk.parse.shiftreduce.ShiftReduceParser(grammar6, trace=3)
rd_parser = nltk.parse.recursivedescent.RecursiveDescentParser(grammar1)
for tree in rd_parser.parse(sent):
  print(tree)
  length = len(tree)
  print(length)
  # while length >= 0:
  #   length = length - 1
  #   for tree in tree:      
  #     # if tree[0] == 'what':
  #     #       print(tree[0] + ' ' 'is')
  #     # else:
  #     #       print(tree[0])  
  #     print(tree[0])

  for tree in tree:
    # print(tree[0])
    for tree in tree:
      # print(tree[0])
      for tree in tree:
        if tree == 'what':
          print(tree + ' ' 'is')
        else:
          print(tree)  

  # list = (tree)
  # print(list)
  # print(list[0])
# nltk.parse.shiftreduce.demo()
# nltk.parse.recursivedescent.demo()

(S (O (SP (PRP you) (NN name))) (VP (QP (WP what))))
2
(PRP you)
(NN name)
(WP what)


In [ ]:
from textblob import TextBlob

text = "Get ball"

blob = TextBlob(text)
blob.tags           # [('The', 'DT'), ('titular', 'JJ'),
                    #  ('threat', 'NN'), ('of', 'IN'), ...]

[('Get', 'VB'), ('the', 'DT'), ('ball', 'NN')]

In [ ]:
nltk.download('treebank')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True